# Data analysis for single particle simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import interpolate
from utils import *

%load_ext autoreload
%autoreload 2

plt.rcParams.update({'font.size': 16})

def gaussian(x, A, sig):
    return A * np.exp(-x**2 / 2 / sig ** 2)

def plot_traj(x,px,s,plot_plasma_density = False):
    if plot_plasma_density:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,figsize=(20, 4))
    else: 
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(15, 4))
    ax1.plot(x,px)
    ax1.set_xlabel('$x\;\;(c/\omega_{p0})$')
    ax1.set_ylabel('$p_x$')
    ax2.plot(s, x)
    ax2.set_xlabel('$z\;\;(c/\omega_{p0})$')
    ax2.set_ylabel('$x\;\;(c/\omega_{p0})$')
    ax3.plot(s, px)
    ax3.set_xlabel('$z\;\;(c/\omega_{p0})$')
    ax3.set_ylabel('$p_x$')
    
    if plot_plasma_density:
        with open('sp_input.json') as f:
            input_file = json.load(f,object_pairs_hook=OrderedDict)
        s_input = input_file['plasma']['s']
        fs_input = input_file['plasma']['fs']
        ax4.plot(s_input,fs_input)
        ax4.set_xlabel('$z\;\;(c/\omega_{p0})$')
        ax4.set_ylabel('$n(z) / n_0$')
        for i in [s[0],s[-1]]:
            ax4.axvline(x=i,color = 'r',linestyle = '--')
            
    fig.tight_layout()
    plt.show()

data_analysis_savedir = 'sp_data_analysis'
if not os.path.isdir(data_analysis_savedir):
    os.mkdir(data_analysis_savedir)

## Load data from input file and output file

In [ ]:
input_file = load_json_file('sp_input.json')
dimension = input_file['simulation']['dimension']
delta_s_dump = input_file['diag']['delta_s_dump']
gamma_init = np.mean(input_file['beam']['gamma'])


output_file = load_json_file('sp_output.json')
x_all = np.array(output_file['x'])
px_all = np.array(output_file['px'])
if dimension == 2:
    y_all = np.array(output_file['y'])
    py_all = np.array(output_file['py'])
s = np.array(output_file['s'])
fs = np.array(output_file['fs'])
gamma_gain = np.array(output_file['gamma_gain'])
gamma_s = gamma_init + gamma_gain
A = np.array(output_file['A'])
sig_i = np.array(output_file['sig_i'])

print(x_all.shape)

In [ ]:
plt.plot(s,fs)
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel('$n(z) / n_0$')
plt.savefig(data_analysis_savedir + '/plasma_density_profile.png',bbox_inches = 'tight',facecolor='w')
plt.show()
print(max(fs))

## 1. Single particle trajectory analysis
### Select the particle index

In [ ]:
idx = 0

### The entire trajectory:

In [ ]:
x = x_all[idx]
px = px_all[idx]

plot_traj(x,px,s)

### Select a section of the trajectory

In [ ]:
idx_start = -30000
idx_end = -1

In [ ]:
x_section = x[idx_start:idx_end]
px_section = px[idx_start:idx_end]
s_section = s[idx_start:idx_end]

plot_traj(x_section,px_section,s_section)

### Extract one period and get phase space area (action)

In [ ]:
x_one_period,px_one_period,s_one_period = get_one_period(x_section,px_section,s_section)
plot_traj(x_one_period,px_one_period,s_one_period)
print('The period is:',s_one_period[-1] - s_one_period[0])
print('The action is:',get_action(x_one_period,px_one_period))

### Plot periodic motion in different parts

In [ ]:
idx_start1 = 0
idx_end1 = 30000

x_section1 = x[idx_start1:idx_end1]
px_section1 = px[idx_start1:idx_end1]
s_section1 = s[idx_start1:idx_end1]

x_one_period1,px_one_period1,s_one_period1 = get_one_period(x_section1,px_section1,s_section1)

plot_traj(x_one_period1,px_one_period1,s_one_period1,True)

print('The action is:',get_action(x_one_period1,px_one_period1))

In [ ]:
idx_start2 = 58000
idx_end2 = 68000

x_section2 = x[idx_start2:idx_end2]
px_section2 = px[idx_start2:idx_end2]
s_section2 = s[idx_start2:idx_end2]

x_one_period2,px_one_period2,s_one_period2 = get_one_period(x_section2,px_section2,s_section2)

plot_traj(x_one_period2,px_one_period2,s_one_period2,True)

print('The action is:',get_action(x_one_period2,px_one_period2))

In [ ]:
idx_start3 = -30000
idx_end3 = -1

x_section3 = x[idx_start3:idx_end3]
px_section3 = px[idx_start3:idx_end3]
s_section3 = s[idx_start3:idx_end3]

x_one_period3,px_one_period3,s_one_period3 = get_one_period(x_section3,px_section3,s_section3)

plot_traj(x_one_period3,px_one_period3,s_one_period3,True)

print('The action is:',get_action(x_one_period3,px_one_period3))

In [ ]:
plt.plot(x_one_period1,px_one_period1,
         label = str(int(s_one_period1[0])) + ' < s <' + str(int(s_one_period1[-1])) 
         + r'$,\lambda_\beta = $' + str(int(s_one_period1[-1]-s_one_period1[0])))
plt.plot(x_one_period2,px_one_period2,
         label = str(int(s_one_period2[0])) + ' < s <' + str(int(s_one_period2[-1])) 
         + r'$,\lambda_\beta = $' + str(int(s_one_period2[-1]-s_one_period2[0])))
plt.plot(x_one_period3,px_one_period3,
         label = str(int(s_one_period3[0])) + ' < s <' + str(int(s_one_period3[-1])) 
         + r'$,\lambda_\beta = $' + str(int(s_one_period3[-1]-s_one_period3[0])))
plt.legend(loc = (1.04,0))
plt.xlabel('$x\;\;(c/\omega_{p0})$')
plt.ylabel('$p_x$')
plt.show()

### Get the final $\lambda_\beta$ and action

In [ ]:
idx_start = -30000
idx_end = -1
N = len(x_all)
x_init_all = x_all[:,0]
lambda_beta_all = []
J_all = []
for idx in range(N):
    x = np.array(x_all[idx])
    px = np.array(px_all[idx])
    x_section = x[idx_start:idx_end]
    px_section = px[idx_start:idx_end]
    s_section = s[idx_start:idx_end]
    x_one_period,px_one_period,s_one_period = get_one_period(x_section,px_section,s_section)
    lambda_beta = s_one_period[-1] - s_one_period[0]
    lambda_beta_all.append(lambda_beta)
    J = get_action(x_one_period,px_one_period)
    J_all.append(J)

In [ ]:
plt.figure(1)
plt.plot(x_init_all,lambda_beta_all,'-o')
plt.xlabel('$x_{init}$')
plt.ylabel(r'$\lambda_\beta$')
plt.show()

plt.figure(2)
plt.plot(x_init_all,J_all,'-o')
plt.xlabel('$x_{init}$')
plt.ylabel('$J$')
plt.show()

In [ ]:
data_save = {}
data_save['lambda_beta'] = lambda_beta_all
data_save['action'] = J_all

save_to_json_file(data_save,'lambda_and_action.json')

## 2. Particle phase space ring analysis

In [ ]:
### some auxiliary calculation
emittance_init = np.sqrt(x_all[:,0].std() **2 * px_all[:,0].std() ** 2 - np.mean(x_all[:,0] * px_all[:,0]) **2)
emitgeo_init = emittance_init / gamma_init
beta_m0 = np.sqrt(2 * gamma_init)
sigma_m0 = np.sqrt(beta_m0 * emitgeo_init)

sigma_m = sigma_m0 / fs ** (1/4) * (gamma_init / gamma_s) ** (1/4)
sigma_p_m = emittance_init / sigma_m

### Plot the evolution of phase space rings

In [ ]:
idx_ranges = [[800,1000],[1000,1200],[1200,1400],[1400,1600]]
timeSteps = list(range(x_all.shape[1]))
normalize = True
# ========================================
if normalize:
    savedir = data_analysis_savedir + '/normalized_phase_space_particle_idx_[' + str(idx_ranges[0][0]) + ','+ str(idx_ranges[-1][-1]) + ')'
else:
    savedir = data_analysis_savedir + '/phase_space_particle_idx_[' + str(idx_ranges[0][0]) + ','+ str(idx_ranges[-1][-1]) + ')'
if not os.path.isdir(savedir):
    os.mkdir(savedir)


for i,timeStep in enumerate(timeSteps):
    for j in range(len(idx_ranges)):
        idx_start,idx_end = idx_ranges[j]
        x_ring = x_all[idx_start:idx_end,timeStep]
        px_ring = px_all[idx_start:idx_end,timeStep]
        color = 'C' + str(j)
    
        if normalize:
            plt.scatter(x_ring / sigma_m[i],px_ring / sigma_p_m[i],s=1,color=color)
            plt.xlabel(r'$x \;\;[(2/\gamma_i)^{1/4} (\epsilon_{ni} c/\omega_{p})^{1/2}]$')
            plt.ylabel(r'$p_x \;\;[(\gamma_i/2)^{1/4} (\epsilon_{ni} \omega_{p}/c)^{1/2}]$')  
            plt.axis('square')
            plt.xlim([-8,8])
            plt.ylim([-8,8])
            plt.xticks(ticks=[-8,-4,0,4,8])
            plt.yticks(ticks=[-8,-4,0,4,8])
        else:
            plt.scatter(x_ring,px_ring,s=1,color=color)
            plt.xlabel('$x\;\;(c/\omega_{p0})$')
            plt.ylabel('$p_x$')
        
    
    plt.title('z = '+ str(timeStep * delta_s_dump) + r'$\;\;(c/\omega_{p0})$' + r'$, n(z)/n_0 = $' + str(round(fs[i],3)))
    plt.savefig(savedir + '/phase_space_' + str(timeStep).zfill(8) + '.png',bbox_inches = 'tight',facecolor='w')
    plt.close()

In [ ]:
particle_idx_start = 0
particle_idx_end = 200
timeStep = 0

x_ring = x_all[particle_idx_start:particle_idx_end][:,timeStep]
px_ring = px_all[particle_idx_start:particle_idx_end][:,timeStep]

plt.scatter(x_ring,px_ring,s=1)
plt.show()

In [ ]:
vx_ring = px_ring / 20000
lambda_beta = get_time(x_ring,vx_ring)
action = abs(get_action(x_ring,px_ring))
print('lambda_beta calculated from a phase space ring is:',lambda_beta)
print('action calculated from a phase space ring is:',action)

In [ ]:
timeSteps = list(range(0,int(s[-1])+1,2000)) 
# ==============================================
from collections import defaultdict
lambda_beta_from_ring = defaultdict(list)
action_from_ring = defaultdict(list)
# select a phase space ring
for ring_idx in range(10):
    particle_idx_start = ring_idx * 200
    particle_idx_end = particle_idx_start + 200
    x_ring_s = x_all[particle_idx_start:particle_idx_end]
    px_ring_s = px_all[particle_idx_start:particle_idx_end]
    # Select a time step
    for timeStep in timeSteps:
        x_ring = x_ring_s[:,timeStep]
        px_ring = px_ring_s[:,timeStep]
        vx_ring = px_ring / 20000  ### Careful: hard-coded! 
        lambda_beta = get_time(x_ring,vx_ring)
        action = abs(get_action(x_ring,px_ring))
        lambda_beta_from_ring[ring_idx].append(lambda_beta)
        action_from_ring[ring_idx].append(action)

save_to_json_file({'s':[delta_s_dump * timeStep for timeStep in timeSteps],\
                   'lambda_beta_from_ring':lambda_beta_from_ring,\
                   'action_from_ring':action_from_ring},data_analysis_savedir + '/lambda_and_action_from_ring.json')

In [ ]:
data = load_json_file(data_analysis_savedir + '/lambda_and_action_from_ring.json')

In [ ]:
ring_idx = '9'
plt.figure(1)
plt.plot(data['s'],data['lambda_beta_from_ring'][ring_idx],'-o')
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel(r'$\lambda_\beta$')

plt.figure(2)
plt.plot(data['s'],data['lambda_beta_from_ring'][ring_idx],'-o')
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel(r'$\lambda_\beta$')
plt.show()

## 3. Whole beam analysis

In [ ]:
beam_data_x = beam_raw_data_analysis(x_all,px_all)
if dimension == 2:
    beam_data_y = beam_raw_data_analysis(y_all,py_all)
beam_data_x.keys()

In [ ]:
# Add comparison to QPAD simulation
# parameters = load_json_file('../JupyterQpic/beam_slices/beam2_[-1.0,-0.5,0.0,0.5,1.0]_0.1')
parameters = load_json_file('../JupyterQpic/beam_slices/beam2_[0.05,0.5,1.0,1.5,1.95]_0.05')
parameters.keys()

xi_slice = '1.5'
parameters[xi_slice].keys()

In [ ]:
normalized_to_initial_emittance = True
compare_to_QPAD = False

if normalized_to_initial_emittance:
    plt.plot(s,np.array(beam_data_x['emittance'])/beam_data_x['emittance'][0],label = 'single particle simulation (x)')
    if dimension == 2:
        plt.plot(s,np.array(beam_data_y['emittance'])/beam_data_y['emittance'][0],'--',label = 'single particle simulation (y)')
    if compare_to_QPAD:
        plt.plot(parameters[xi_slice]['s'],np.array(parameters[xi_slice]['epsilon_n_x']) / parameters[xi_slice]['epsilon_n_x'][0],label = 'QPAD (x)')
    plt.ylabel('$\epsilon_n / \epsilon_{ni}$')
else:
    plt.plot(s,beam_data_x['emittance'],label = 'single particle simulation (x)')
    if dimension == 2:
        plt.plot(s,beam_data_y['emittance'],label = 'single particle simulation (y)')
    if compare_to_QPAD:
        plt.plot(parameters[xi_slice]['s'],parameters[xi_slice]['epsilon_n_x'],label = 'QPAD (x)')
    plt.ylabel('$\epsilon_n \;\;(c/\omega_{p0})$')
    
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.legend(loc = (1.04,0))

if normalized_to_initial_emittance:
    plt.savefig(data_analysis_savedir + '/emittance_over_initial_emittance.png',bbox_inches = 'tight',facecolor='w')
else:
    plt.savefig(data_analysis_savedir + '/emittance.png',bbox_inches = 'tight',facecolor='w')

plt.show()

In [ ]:
plt.plot(s,beam_data_x['sigma'],label = 'single particle simulation (x)')
if dimension == 2:
    plt.plot(s,beam_data_y['sigma'],'--',label = 'single particle simulation (y)')
plt.plot(s,sigma_m,'--',label = '$\\sigma_m$')
if compare_to_QPAD:
    plt.plot(parameters[xi_slice]['s'],parameters[xi_slice]['sigma_x'],'--',label = 'QPAD (x)')
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel('$\\sigma \;\;(c/\omega_{p0})$')
plt.legend(loc = (1.04,0))
plt.savefig(data_analysis_savedir + '/sigma.png',bbox_inches = 'tight',facecolor='w')
plt.show()

In [ ]:
alpha_x = -np.array(beam_data_x['x_times_px_avg']) / np.array(beam_data_x['emittance'])
plt.plot(s,alpha_x,label = 'single particle simulation (x)')
if dimension == 2:
    alpha_y = -np.array(beam_data_y['x_times_px_avg']) / np.array(beam_data_y['emittance'])
    plt.plot(s,alpha_y,'--',label = 'single particle simulation (y)')
if compare_to_QPAD:
    plt.plot(parameters[xi_slice]['s'],parameters[xi_slice]['alpha_x'],'--',label = 'QPAD (x)')
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel('$\\alpha$')
plt.legend(loc = (1.04,0))
plt.savefig(data_analysis_savedir + '/alpha.png',bbox_inches = 'tight',facecolor='w')
plt.show()

## Phase space scatter plot

In [ ]:
dir_save_phase_space = 'phase_space'

if not os.path.isdir(dir_save_phase_space):
    os.mkdir(dir_save_phase_space)
    
s_dump = input_file['diag']['delta_s_dump']

epsilon_over_epsilon_init = np.array(beam_data_x['emittance'])/beam_data_x['emittance'][0]

for i in range(x_all.shape[1]):
    plt.scatter(x_all[:,i],px_all[:,i],s=1)
    plt.xlabel(r'$x \;\;(c/\omega_{p0})$')
    plt.ylabel('$p_x$')
    plt.title('z = ' + str(int(i * s_dump))+'$\;\;(c/\omega_{p0})$' + '$,\; \epsilon_n / \epsilon_{ni} = $' + str(round(epsilon_over_epsilon_init[i],3)))
    plt.savefig(dir_save_phase_space + '/phase_space_s='+str(int(i * s_dump)).zfill(8)+'.png',bbox_inches = 'tight',facecolor='w')
    plt.close()

## Plot ion collapse model and acceleration model

In [ ]:
savedir = data_analysis_savedir + '/ion_model'
x = np.linspace(-0.1,0.1,100)
s_dump = np.arange(x_all.shape[1]) * delta_s_dump

#######################################################

if not os.path.isdir(savedir):
    os.mkdir(savedir)

for i in range(len(s_dump)):

    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15, 4))
    ax1.plot(s,fs)
    ax1.axvline(x=s_dump[i],linestyle = '--',color = 'r')
    ax1.set_xlabel('$z\;\;(c/\omega_{p0})$')
    ax1.set_ylabel('$n(z) / n_0$')
    ax1.set_title('z = ' + str(round(s[i])) + ', $n(z)/n_0 =$' + str(round(fs[i],4)))
    
    ni = fs[i] + gaussian(x, A[i], sig_i[i])
    ax2.plot(x,ni)
    ax2.set_xlabel('$x\;\;(c/\omega_{p0})$')
    ax2.set_ylabel('$n_i(x)$')
    ax2.set_title('A = '+str(round(A[i],1)) + ', $\\sigma_{ion} =$' + str(round(sig_i[i],3)))
    
    plt.savefig(savedir + '/ion_collapse_model_z=' + str(round(s[i])).zfill(8) +'.png',bbox_inches = 'tight',facecolor='w')
    # ax2.set_ylim([0,np.max(A)])
    plt.close()

In [ ]:
# A is normalized to plasma density at the density plateau
plt.plot(s,A)
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel('$A \;\;(n_0)$')
plt.savefig(data_analysis_savedir + '/A.png',bbox_inches = 'tight',facecolor='w')
plt.show()

In [ ]:
plt.plot(s,sig_i)
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel('$\\sigma_{ion} \;\;(c/\omega_{p0})$')
plt.savefig(data_analysis_savedir + '/sigma_ion.png',bbox_inches = 'tight',facecolor='w')
plt.show()

In [ ]:
plt.plot(s,gamma_s)
plt.xlabel('$z\;\;(c/\omega_{p0})$')
plt.ylabel('$\gamma$')
plt.savefig(data_analysis_savedir + '/gamma.png',bbox_inches = 'tight',facecolor='w')
plt.show()

### Select a phase space contour from whole beam

In [ ]:
# beta_init = beam_data_x['sigma'][0] ** 2 / (beam_data_x['emittance'][0] / gamma_init)
# alpha_init = alpha_x[0]
# gamma_Twiss_init = (1 + alpha_init ** 2) / beta_init
# z_relative_to_vacuum_focus_init = - alpha_init / gamma_Twiss_init

# beta_final = beam_data_x['sigma'][-1] ** 2 / (beam_data_x['emittance'][-1] / gamma_s[-1])
# alpha_final = alpha_x[-1]
# gamma_Twiss_final = (1 + alpha_final ** 2) / beta_final
# z_relative_to_vacuum_focus_final = - alpha_final / gamma_Twiss_final

In [ ]:
def propagate_to_vacuum_focus(x,px,gamma=1,alpha=None,beta=None,gamma_Twiss=None):
    if alpha != None:
        if gamma_Twiss == None:
            if beta == None:
                print('Wrong input! Return x and xp from input!')
                return x,px
            else:
                gamma_Twiss = (1 + alpha ** 2) / beta
                
        z_relative_to_vacuum_focus = - alpha / gamma_Twiss
        px_at_vacuume_focus = np.array(px)
        x_at_vacuum_focus = x - px_at_vacuume_focus / gamma * z_relative_to_vacuum_focus
        return x_at_vacuum_focus,px_at_vacuume_focus
    else:
        x = x - np.mean(x)
        x_prime = px / gamma
        x_prime = x_prime - np.mean(x_prime)
        
        x2_avg = np.mean(x ** 2)
        x_prime2_avg = np.mean(x_prime ** 2)
        x_times_x_prime_avg = np.mean(x * x_prime)
        emitgeo_x = np.sqrt(x2_avg * x_prime2_avg - x_times_x_prime_avg ** 2)
        
        alpha = - x_times_x_prime_avg / emitgeo_x
        gamma_Twiss = x_prime2_avg / emitgeo_x
        
        z_relative_to_vacuum_focus = - alpha / gamma_Twiss
        px_at_vacuume_focus = np.array(px)
        x_at_vacuum_focus = x - px_at_vacuume_focus / gamma * z_relative_to_vacuum_focus

        return x_at_vacuum_focus,px_at_vacuume_focus

In [ ]:
# R = 0.3
dR = 0.01

_,num_s_step = x_all.shape
middle_s_step = num_s_step // 2
last_s_step = num_s_step - 1


for R in [1]:#[0.01,0.02,0.05,0.1,0.2,0.5,1,2]:

    ### Select the particles on the same phase space contour
    
    ### If the initial alpha is not 0, we need to transform the tilted phase space ellipse into a horizontal & vertical phase space ellipse
    ### so that x and x' are separable
    
    x_init_at_vacuum_focus, px_init_at_vacuum_focus = propagate_to_vacuum_focus(x_all[:,0],px_all[:,0],gamma = gamma_s[0])
    x_init_at_vacuum_focus_normalized = x_init_at_vacuum_focus / sigma_m[0]
    px_init_at_vacuum_focus_normalized = px_init_at_vacuum_focus / sigma_p_m[0]
    
    
    x_middle_at_vacuum_focus, px_middle_at_vacuum_focus = propagate_to_vacuum_focus(x_all[:,middle_s_step],px_all[:,middle_s_step],gamma = gamma_s[middle_s_step])
    x_middle_at_vacuum_focus_normalized = x_middle_at_vacuum_focus / sigma_m[middle_s_step]
    px_middle_at_vacuum_focus_normalized = px_middle_at_vacuum_focus / sigma_p_m[middle_s_step]
    
    
    x_final_at_vacuum_focus, px_final_at_vacuum_focus = propagate_to_vacuum_focus(x_all[:,-1],px_all[:,-1],gamma = gamma_s[-1])
    x_final_at_vacuum_focus_normalized = x_final_at_vacuum_focus / sigma_m[-1]
    px_final_at_vacuum_focus_normalized = px_final_at_vacuum_focus / sigma_p_m[-1]
    
    ### Select particles belong to the same ring

    normalized_phase_space_radius_init = np.sqrt(x_init_at_vacuum_focus_normalized ** 2 + px_init_at_vacuum_focus_normalized ** 2)
    in_ring = (normalized_phase_space_radius_init < (R + dR)) & (normalized_phase_space_radius_init > (R - dR))
    

    ### Plot the particles on this contour at the beginning, in the middle, and at the end

    fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2, 3,figsize=(18, 10))

    ax1.scatter(x_all[in_ring,0],px_all[in_ring,0],s=1) 
    ax1.set_xlabel('$x \; (c/\omega_{p0})$')
    ax1.set_ylabel('$p_x$')
    ax1.set_title('$s = 0 \;\; (c/\omega_{p0})$')
    
    ax2.scatter(x_all[in_ring,middle_s_step],px_all[in_ring,middle_s_step],s=1) 
    ax2.set_xlabel('$x \; (c/\omega_{p0})$')
    ax2.set_ylabel('$p_x$')
    ax2.set_title('$s = $' + str(int(middle_s_step * delta_s_dump)) + '$ \;\; (c/\omega_{p0})$')
    
    ax3.scatter(x_all[in_ring,last_s_step],px_all[in_ring,last_s_step],s=1) 
    ax3.set_xlabel('$x \; (c/\omega_{p0})$')
    ax3.set_ylabel('$p_x$')
    ax3.set_title('$s = $' + str(int(last_s_step * delta_s_dump)) + '$ \;\; (c/\omega_{p0})$')

    ax4.scatter(x_init_at_vacuum_focus_normalized[in_ring],px_init_at_vacuum_focus_normalized[in_ring],s=1) 
    ax4.set_xlabel('$x_{vf}/\\sigma_m$')
    ax4.set_ylabel('$p_{x,vf} / \\sigma_{pm}$')
    ax4.set_title('$s = 0 \;\; (c/\omega_{p0})$')
    
    ax5.scatter(x_middle_at_vacuum_focus_normalized[in_ring],px_middle_at_vacuum_focus_normalized[in_ring],s=1) 
    ax5.set_xlabel('$x_{vf}/\\sigma_m$')
    ax5.set_ylabel('$p_{x,vf} / \\sigma_{pm}$')
    ax5.set_title('$s = $' + str(int(middle_s_step * delta_s_dump)) + '$ \;\; (c/\omega_{p0})$')

    ax6.scatter(x_final_at_vacuum_focus_normalized[in_ring],px_final_at_vacuum_focus_normalized[in_ring],s=1) 
    ax6.set_xlabel('$x_{vf}/\\sigma_m$')
    ax6.set_ylabel('$p_{x,vf} / \\sigma_{pm}$')
    ax6.set_title('$s = $' + str(int(last_s_step * delta_s_dump)) + '$ \;\; (c/\omega_{p0})$')
    

    if not os.path.isdir(data_analysis_savedir + '/phase_space_rings'):
        os.mkdir(data_analysis_savedir + '/phase_space_rings')

    plt.tight_layout()
    plt.savefig(data_analysis_savedir + '/phase_space_rings/R='+str(R)+'_dR='+str(dR)+'.png',bbox_inches = 'tight',facecolor='w')

    plt.show()

In [ ]:
    # x_ring_init_normalized = x_all_ring[:,0] / sigma_m[0]
    # px_ring_init_normalized = px_all_ring[:,0] / sigma_p_m[0]

    # R_ring_init = np.sqrt(x_ring_init_normalized ** 2 + px_ring_init_normalized ** 2)
    # R_mean_init = np.mean(R_ring_init)
    # R_std_init = np.std(R_ring_init)
    
    
    # ax4.set_title('$\\bar R = $'+str(round(R_mean_init,3)) + '$,\\sigma_R = $' + str(round(R_std_init,3)) + '\n'+ \
    #              '$s = 0 \;\; (c/\omega_{p0})$')

    # R_ring_last = np.sqrt(x_ring_last_normalized ** 2 + px_ring_last_normalized ** 2)
    # R_mean_last = np.mean(R_ring_last)
    # R_std_last = np.std(R_ring_last)